# DOSTA Calibration Parser

This script and notebook gives the code for loading the CTD calibration coefficients into a properly named calibration csv, as well as an example of how to use it. The calibration coefficients can be parsed from either the vendor calibration (.cal) file, the vendor xmlcon file, or from the capture file (either .cap, .log, or .txt) from the QCT check-in.


In [6]:
from utils import *

In [10]:
import csv
import datetime
import os
import shutil
import sys
import time
import re
import xml.etree.ElementTree as et
import pandas as pd
from zipfile import ZipFile

In [12]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD'

In [7]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [9]:
DOFST = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='DOFST',whoi=True)
DOFST

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
401,DOSTA,D,126,115287,A00176,CGINS-DOSTAD-00126,4831,1336-00002-00004,5.0.4,Aanderaa,...,3305-00115-00128\n3305-00115-00234\n3305-00115...,NaN,NaN,3305-00900-00082\n3305-00900-00320,WH-SC12-03-D02-1002,01/28/2013,GI03FLMB-00001\nCP03ISSM-00005,CP04OSSM-00009,NSIF,NaN
402,DOSTA,D,127,115280\n118157,A00169\nA02064,CGINS-DOSTAD-00127,4831,1336-00002-00004,4.9.1,Aanderaa,...,3305-00115-00109\n3305-00115-00291,NaN,NaN,3305-00900-00059\n3305-00900-00406,WH-SC12-03-D02-1001,01/24/2013,GP03FLMA-00002\nGP03FLMB-00005,NaN,NaN,PAPA FLM(?)
403,DOSTA,D,128,115281,A00170,CGINS-DOSTAD-00128,4831,1336-00002-00004,NaN,Aanderaa,...,3305-00115-00110\n3305-00115-00232,NaN,NaN,3305-00900-00059\n3305-00900-00261,WH-SC12-03-D02-1001,01/24/2013,GP03FLMB-00001\nGP03FLMB-00004\nCP01CNSM-00009,NaN,NaN,NaN
404,DOSTA,D,129,115288\n118304,A00177\nA02162,CGINS-DOSTAD-00129,4831,1336-00002-00004,4.9.1,Aanderaa,...,3305-00115-00113\n3305-00115-00292,NaN,NaN,3305-00900-00059\n3305-00900-00406,WH-SC12-03-D02-1002,01/28/2013,GP03FLMA-00001\nGP03FLMA-00005,NaN,NaN,PAPA FLM(?)
405,DOSTA,D,130,115276,A00166,CGINS-DOSTAD-00130,4831,1336-00002-00004,4.5.7,Aanderaa,...,3305-00115-00005\n3305-00115-00120\n3305-00115...,NaN,NaN,3305-00900-00001\n3305-00900-00070\n3305-00900...,WH-SC12-03-D02-1001,2013-01-24 00:00:00,CP03ISSM-00002\nGP03FLMA-00004\nCP04OSSM-00008,NaN,NaN,NaN
406,DOSTA,D,131,115277,A00167,CGINS-DOSTAD-00131,4831,1336-00002-00004,4.9.1,Aanderaa,...,3305-00115-00006\n3305-00115-00071\n3305-00115...,NaN,NaN,3305-00900-00007\n3305-00900-00353,WH-SC12-03-D02-1001,2013-01-24 00:00:00,CP01CNSM-00001\nGA/GS spare\nGS03FLMA-00003,NaN,NaN,NaN
407,DOSTA,D,132,115278,A00168,CGINS-DOSTAD-00132,4831,1336-00002-00004,4.5.7,Aanderaa,...,3305-00115-00007\n3305-00115-00072,NaN,NaN,3305-00900-00007,WH-SC12-03-D02-1001,41298,CP01CNSM-00001\nGS01SUMO-00002,NaN,Inductive Wire (40),Lost on GS01SUMO-00002 (40m depth)
408,DOSTA,D,211,115703,A00397,CGINS-DOSTAD-00211,4831,1336-00002-00004,4.5.7,Aanderaa,...,3305-00115-00022\n3305-00115-00112\n3305-00115...,NaN,NaN,3305-00900-00059\n3305-00900-00281,WH-SC12-03-D02-1003,2013-08-26 00:00:00,GI01SUMO-00001\nGI01SUMO-00003\nGI01SUMO-00005,GI01SUMO-00005,Bottom of Buoy,GI01SUMO-00001 (buoy)
409,DOSTA,D,212,115702,A00396,CGINS-DOSTAD-00212,4831,1336-00002-00004,4.9.1,Aanderaa,...,3305-00115-00023\n3305-00115-00127\n3305-00115...,NaN,NaN,3305-00900-00082\n3305-00900-00281,WH-SC12-03-D02-1003,2013-08-26 00:00:00,GI01SUMO-00001\nGI01SUMO-00003\nGI01SUMO-00005,GI01SUMO-00005,NSIF,GI01SUMO-00001 (NSIF)
410,DOSTA,D,213,115701,A00395,CGINS-DOSTAD-00213,4831,1336-00002-00004,4.5.7,Aanderaa,...,3305-00115-00024,NaN,NaN,NaN,WH-SC12-03-D02-1003,2013-08-26 00:00:00,NaN,NaN,NaN,"Taken on Pioneer 1 as spare, loaned to Pioneer..."


### What does the DOSTA Calibration CSV file format look like?

In [35]:
# Import/find the location of a DOSTA csv file
csv_dict = load_asset_management(DOSTA, asset_management_directory)
csv_dict

{'CGINS-DOSTAD-00126': ['CGINS-DOSTAD-00126__20180309.csv',
  'CGINS-DOSTAD-00126__20170819.csv',
  'CGINS-DOSTAD-00126__20161011.csv',
  'CGINS-DOSTAD-00126__20140916.csv'],
 'CGINS-DOSTAD-00127': ['CGINS-DOSTAD-00127__20140617.csv',
  'CGINS-DOSTAD-00127__20180920.csv',
  'CGINS-DOSTAD-00127__20151206.csv'],
 'CGINS-DOSTAD-00128': ['CGINS-DOSTAD-00128__20130724.csv',
  'CGINS-DOSTAD-00128__20160704.csv',
  'CGINS-DOSTAD-00128__20170911.csv'],
 'CGINS-DOSTAD-00129': ['CGINS-DOSTAD-00129__20180920.csv',
  'CGINS-DOSTAD-00129__20150612.csv',
  'CGINS-DOSTAD-00129__20130721.csv'],
 'CGINS-DOSTAD-00130': ['CGINS-DOSTAD-00130__20150509.csv',
  'CGINS-DOSTAD-00130__20160701.csv',
  'CGINS-DOSTAD-00130__20170911.csv'],
 'CGINS-DOSTAD-00131': ['CGINS-DOSTAD-00131__20131121.csv',
  'CGINS-DOSTAD-00131__20180524.csv',
  'CGINS-DOSTAD-00131__20161201.csv'],
 'CGINS-DOSTAD-00132': ['CGINS-DOSTAD-00132__20151214.csv',
  'CGINS-DOSTAD-00132__20131121.csv'],
 'CGINS-DOSTAD-00211': ['CGINS-DOSTAD-002

In [37]:
uids = list(csv_dict.keys())

In [38]:
csv_files = csv_dict[uids[0]]

In [39]:
csv_files

['CGINS-DOSTAD-00126__20180309.csv',
 'CGINS-DOSTAD-00126__20170819.csv',
 'CGINS-DOSTAD-00126__20161011.csv',
 'CGINS-DOSTAD-00126__20140916.csv']

In [46]:
pd.read_csv('/'.join((asset_management_directory,csv_files[0])))

,serial,name,value,notes
0,126,CC_conc_coef,"[0.0, 1.0]","[offset, slope]"
1,126,CC_csv,"[0.00268474, 0.000117084, 2.23543e-06, 198.627...",NaN


In [47]:
asset_management_directory

'/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD'

### What does the DOSTA Calibration file format look like?

1. The calibration files are all pdf files sent by SeaBird. They are most likely NOT machine readable, which presents a challenge in automating this approach.

2. Conclusion: I won't try to read the pdfs. Instead, I'll focus on writing the csv cal sheet from the qct checkin info.

['DOSTA_Optode-4831_SN_391_Multipoint_Calibration_2014-07-24.pdf',
 'DOSTA-J_Optode-4831_SN_209_Calibration_Certificate_2017-09-05.zip',
 'DOSTA_Optode-4831_SN-136-218-221-313_repair_report_2018-02-07.pdf',
 'DOSTA_Optode-4831_SN-217-219-291-477-482-484-485_repair_report_2017-06-05.pdf',
 'DOSTA_Optode-4831_SN_126_Calibration_Certificate_2016-03-11.pdf',
 'DOSTA_Optode-4831_SN_126_Calibration_Certificate_2017-08-19.pdf',
 'DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2013-02-27.pdf',
 'DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2018-03-09.pdf',
 'DOSTA_Optode-4831_SN_127_Calibration_Certificate_2018-09-20.pdf',
 'DOSTA_Optode-4831_SN_127_Multipoint_Calibration_2013-03-04.pdf',
 'DOSTA_Optode-4831_SN_127_Multipoint_Calibration_2016-02-01.pdf',
 'DOSTA_Optode-4831_SN_127_Test_Specifications_2013-01-24.pdf',
 'DOSTA_Optode-4831_SN_128_Calibration_Certificate_2017-09-11.pdf',
 'DOSTA_Optode-4831_SN_128_Multipoint_Calibration_2013-03-04.pdf',
 'DOSTA_Optode-4831_SN_128_Multipoint_Ca

### What do the DOSTA QCT file format look like?

In [142]:
#!/usr/bin/env python

import csv
import datetime
import os
import shutil
import sys
import time
import re
import xml.etree.ElementTree as et
import pandas as pd
from zipfile import ZipFile


class DOSTACalibration():
    # Class that stores calibration values for DOSTA's.

    def __init__(self, uid, calibration_date):
        self.serial = ''
        self.uid = uid
        self.date = pd.to_datetime(calibration_date).strftime('%Y%m%d')
        self.coefficients = {'CC_conc_coef':None,
                            'CC_csv':None}
        self.notes = {'CC_conc_coef':None,
                      'CC_csv':None}
                
    @property
    def uid(self):
        return self._uid
        
    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = d.split('-')[2]
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")
            
            
    def generate_file_path(self,dirpath,filename,ext=['.cap','.txt','.log'],exclude=['_V','_Data_Workshop']):
        """
        Function which searches for the location of the given file and returns
        the full path to the file.
        
        Args:
            dirpath - parent directory path under which to search
            filename - the name of the file to search for
            ext - file endings to search for
            exclude - optional list which allows for excluding certain
                directories from the search
        Returns:
            fpath - the file path to the filename from the current
                working directory.
        """
        # Check if the input file name has an extension already
        # If it does, parse it for input into the search algo
        if '.' in filename:
            check = filename.split('.')
            filename = check[0]
            ext = ['.'+check[1]]
        
        for root, dirs, files in os.walk(dirpath):
            dirs[:] = [d for d in dirs if d not in exclude]
            for fname in files:
                if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                    fpath = os.path.join(root, fname)
                    return fpath
                
    def load_qct(self, filepath):
        """
        Function which parses the output from the QCT check-in and loads them into
        the DOSTA object.
        
        Args:
            filepath - the full directory path and filename 
        Raises:
            ValueError - checks if the serial number parsed from the UID matches the
                the serial number stored in the file.
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument 
        """
        
        data = {}
        with open(filepath, errors='ignore') as file:
            reader = csv.reader(file, delimiter='\t')
            for row in reader:
                data.update({reader.line_num:row})
                
        for key,info in data.items():
            # Find the serial number from the QCT check-in and compare to UID
            if 'serial number' in [x.lower() for x in info]:
                serial_num = info[-1].zfill(5)
                if self.serial != serial_num:
                    raise ValueError(f'Serial number {serial_num.zfill(5)} from the QCT file does not match {self.serial} from the UID.')
                else:
                    pass
                
            # Find the svu foil coefficients
            if 'svufoilcoef' in [x.lower() for x in info]:
                self.coefficients['CC_csv'] = [float(n) for n in info[3:]]
            
            # Find the concentration coefficients
            if 'conccoef' in [x.lower() for x in info]:
                self.coefficients['CC_conc_coef'] = [float(n) for n in info[3:]]
                
    def add_notes(self, notes):
        """
        This function adds notes to the calibration csv based on the 
        calibration coefficients.
        
        Args:
            notes - a dictionary with keys of the calibration coefficients
                which correspond to an entry of desired notes about the 
                corresponding coefficients
        Returns:
            self.notes - a dictionary with the entered notes.
        """
        keys = notes.keys()
        for key in keys:
            self.notes[key] = notes[key]


    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.
        
        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary 
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is 
                written to the specified directory from the outpath.
        """
        
        # Run a check that the coefficients have actually been loaded
        for key in self.coefficients.keys():
            if self.coefficients[key] == None:
                raise ValueError(f'No coefficients for {key} have been loaded.')
            
        # Create a dataframe to write to the csv
        data = {'serial':[self.serial]*len(self.coefficients),
               'name':list(self.coefficients.keys()),
               'value':list(self.coefficients.values()),
               'notes':list(self.notes.values()) }
        df = pd.DataFrame().from_dict(data)
        
        # Generate the csv name
        csv_name = self.uid + '__' + self.date + '.csv'
        
        # Now write to 
        check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)

In [143]:
DOSTA = DOSTACalibration(uid='CGINS-DOSTAD-00126',calibration_date='March 22, 2019')

In [144]:
DOSTA.date

'20190322'

In [145]:
filepath = DOSTA.generate_file_path(qct_directory,'3305-00115-00128')
filepath

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/3305-00115-00128-A.log'

In [146]:
DOSTA.load_qct(filepath)

In [147]:
DOSTA.coefficients

{'CC_conc_coef': [0.0, 1.0],
 'CC_csv': [0.002648947,
  0.0001126617,
  2.243911e-06,
  230.6865,
  -0.3189642,
  -54.81301,
  4.567296]}

In [148]:
DOSTA.serial

'00126'

In [149]:
DOSTA.add_notes({'CC_conc_coef':'[intercept,slope]'})

In [150]:
outdir = '/'.join((os.getcwd(),'temp'))

In [151]:
DOSTA.write_csv(outdir)

Write CGINS-DOSTAD-00126__20190322.csv to /home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp? [y/n]: y


In [73]:
out = generate_file_path(asset_management_directory, csv_files[0].split('.')[0],ext=['.csv'])
print(out)

/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD/CGINS-DOSTAD-00126__20180309.csv


In [82]:
check = csv_files[0].split('.')
check

['CGINS-DOSTAD-00126__20180309', 'csv']

In [81]:
if type(check) is str:
    print('This logic works')

This logic works


In [68]:
csv_files[0]

'CGINS-DOSTAD-00126__20180309.csv'

In [16]:
qct_filepath = generate_file_path(qct_directory, '3305-00115-00128')

In [54]:
data = {}
with open(qct_filepath, errors='ignore') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.update({reader.line_num:row})

In [62]:
for key,info in data.items():
    if 'serial number' in [x.lower() for x in info]:
        print(key)
        print(info)

13
['Serial Number', '4831', '126', '126']
62
['Serial Number', '4831', '126', '126']


In [108]:
data

{1: ['=~=~=~=~=~=~=~=~=~=~=~= PuTTY log 2016.03.14 10:58:42 =~=~=~=~=~=~=~=~=~=~=~='],
 2: ['\x11%'],
 3: ['StartupInfo',
  '4831',
  '126',
  'Mode',
  'AADI Smart Sensor Terminal Protocol',
  'RS232 Protocol Version',
  '3',
  'Config Version',
  '12'],
 4: ['\x11MEASUREMENT',
  '4831',
  '126',
  'O2Concentration[uM]',
  '259.580',
  'AirSaturation[%]',
  '97.618',
  'Temperature[Deg.C]',
  '23.370',
  'CalPhase[Deg]',
  '30.176',
  'TCPhase[Deg]',
  '30.176',
  'C1RPh[Deg]',
  '35.159',
  'C2RPh[Deg]',
  '4.983',
  'C1Amp[mV]',
  '992.7',
  'C2Amp[mV]',
  '894.8',
  'RawTemp[mV]',
  '-11.9'],
 5: ['\x13\x11MEASUREMENT',
  '4831',
  '126',
  'O2Concentration[uM]',
  '258.770',
  'AirSaturation[%]',
  '97.309',
  'Temperature[Deg.C]',
  '23.368',
  'CalPhase[Deg]',
  '30.213',
  'TCPhase[Deg]',
  '30.213',
  'C1RPh[Deg]',
  '35.232',
  'C2RPh[Deg]',
  '5.019',
  'C1Amp[mV]',
  '992.7',
  'C2Amp[mV]',
  '893.5',
  'RawTemp[mV]',
  '-11.8'],
 6: ['\x13'],
 7: ['!\x11MEASUREMENT',
  '48

In [63]:
for key,info in data.items():
    if 'svufoilcoef' in [x.lower() for x in info]:
        print(key)
        print(info)

25
['SVUFoilCoef', '4831', '126', '2.648947E-03', '1.126617E-04', '2.243911E-06', '2.306865E+02', '-3.189642E-01', '-5.481301E+01', '4.567296E+00']
92
['SVUFoilCoef', '4831', '126', '2.648947E-03', '1.126617E-04', '2.243911E-06', '2.306865E+02', '-3.189642E-01', '-5.481301E+01', '4.567296E+00']


In [85]:
info[3:]

['101.881',
 '23.434',
 '29.679',
 '29.679',
 '34.921',
 '5.242',
 '974.1',
 '883.3',
 '-13.9']

In [64]:
for key,info in data.items():
    if 'conccoef' in [x.lower() for x in info]:
        print(key)
        print(info)

26
['ConcCoef', '4831', '126', '0.000000E+00', '1.000000E+00']
93
['ConcCoef', '4831', '126', '0.000000E+00', '1.000000E+00']


In [26]:
data

b'=~=~=~=~=~=~=~=~=~=~=~= PuTTY log 2016.03.14 10:58:42 =~=~=~=~=~=~=~=~=~=~=~=\r\n\xff\x11%\r\nStartupInfo\t4831\t126\tMode\tAADI Smart Sensor Terminal Protocol\tRS232 Protocol Version\t3\tConfig Version\t12\r\n\x11MEASUREMENT\t4831\t126\tO2Concentration[uM]\t259.580\tAirSaturation[%]\t97.618\tTemperature[Deg.C]\t23.370\tCalPhase[Deg]\t30.176\tTCPhase[Deg]\t30.176\tC1RPh[Deg]\t35.159\tC2RPh[Deg]\t4.983\tC1Amp[mV]\t992.7\tC2Amp[mV]\t894.8\tRawTemp[mV]\t-11.9\r\n\x13\x11MEASUREMENT\t4831\t126\tO2Concentration[uM]\t258.770\tAirSaturation[%]\t97.309\tTemperature[Deg.C]\t23.368\tCalPhase[Deg]\t30.213\tTCPhase[Deg]\t30.213\tC1RPh[Deg]\t35.232\tC2RPh[Deg]\t5.019\tC1Amp[mV]\t992.7\tC2Amp[mV]\t893.5\tRawTemp[mV]\t-11.8\r\n\x13\r\n!\x11MEASUREMENT\t4831\t126\tO2Concentration[uM]\t258.808\tAirSaturation[%]\t97.321\tTemperatsure[Deg.C]\t23.367\tCalPhase[Deg]\t30.212\tTCPhase[Deg]\t30.212\tC1RPh[Deg]\t35.243\ttC2RPh[Deg]\t5.031\tC1Amp[mV]\t992.6\tC2Amp[mV]\t891.5\tRawTemp[mV]\t-11.8\r\nop\r\n#\r\n